In [1]:
import pandas as pd


In [2]:
df = pd.read_pickle("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")
df

,emotion,dialog,utterance,text,num_words,split,audio_path,name,label,num_channels,audio_shape,size_padding,video_path
11396,4,38,4,"oh it s great , it s a role on",10,test,../../data/test_splits_wav/dia38_utt4.wav,dia38_utt4,joy,6,4879360,29276160,../../data/test_video/dia38_utt4.mp4
12087,5,220,0,what s that smell ?,5,test,../../data/test_splits_wav/dia220_utt0.wav,dia220_utt0,disgust,6,3761152,22566912,../../data/test_video/dia220_utt0.mp4
9943,4,309,0,kathy ! kathy ! hi ! ! kathy ! kathy ! ( she d...,29,train,../../data/train_splits_wav/dia309_utt0.wav,dia309_utt0,joy,6,656725,3940350,../../data/train_video/dia309_utt0.mp4
10195,4,485,0,"hey , this heyy",4,train,../../data/train_splits_wav/dia485_utt0.wav,dia485_utt0,joy,6,412672,2476032,../../data/train_video/dia485_utt0.mp4
3411,0,757,2,"y'know , i do n't know if you 've ever looked ...",50,train,../../data/train_splits_wav/dia757_utt2.wav,dia757_utt2,neutral,6,388437,2330622,../../data/train_video/dia757_utt2.mp4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,1,516,2,oh i m sorry ! do you need a break ?,11,train,../../data/train_splits_wav/dia516_utt2.wav,dia516_utt2,surprise,2,2048,4096,../../data/train_video/dia516_utt2.mp4
7031,1,509,14,ow ! ow ! contraction ow ow ! ow ow !,11,train,../../data/train_splits_wav/dia509_utt14.wav,dia509_utt14,surprise,2,2048,4096,../../data/train_video/dia509_utt14.mp4
10167,4,468,4,i m so glad you could make it,8,train,../../data/train_splits_wav/dia468_utt4.wav,dia468_utt4,joy,2,2048,4096,../../data/train_video/dia468_utt4.mp4
9618,4,103,6,i love hiking !,4,train,../../data/train_splits_wav/dia103_utt6.wav,dia103_utt6,joy,2,2048,4096,../../data/train_video/dia103_utt6.mp4


In [3]:
train_csv = pd.read_csv("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/train_sent_emo.csv")
val_csv = pd.read_csv("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/dev_sent_emo.csv")
test_csv = pd.read_csv("/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/data/test_sent_emo.csv")

In [4]:
val_csv 

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"
...,...,...,...,...,...,...,...,...,...,...,...
1104,1174,No.,Monica,sadness,negative,113,9,6,2,"00:19:28,792","00:19:29,876"
1105,1175,What? Oh my God! Im gonna miss you so much!,Rachel,sadness,negative,113,10,6,2,"00:19:33,213","00:19:35,965"
1106,1176,Im gonna miss you!,Monica,sadness,negative,113,11,6,2,"00:19:36,175","00:19:37,967"
1107,1177,I mean its the end of an era!,Rachel,sadness,negative,113,12,6,2,"00:19:39,094","00:19:40,928"
